https://towardsdatascience.com/data-science-skills-web-scraping-javascript-using-python-97a29738353f

https://exceptionshub.com/selenium-cant-click-element-because-other-element-obscures-it.html

https://www.toolsqa.com/selenium-webdriver/handle-iframes-in-selenium/

https://www.selenium.dev/documentation/webdriver/elements/finders/

https://www.selenium.dev/selenium/docs/api/javascript/module/selenium-webdriver/index_exports_By.html

Add Excel Formatting
https://medium.com/codeptivesolutions/https-medium-com-nensi26-formatting-in-excel-sheet-using-xlsxwriter-part-1-2c2c547b2bea

In [1]:
import time, datetime, requests,random, os, hashlib

from pathlib import Path

from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By

import pandas as pd
#//*** install xlsxwriter for pandas
#pip install xlsxwriter

#//*********************************************************
#//*** Build the tgt_date from the slected Quarter and Year
#//*********************************************************
tgt_year = 2022
quarter = "Q4"

if quarter == "Q1":
    tgt_date = f"{tgt_year}-01-01"
elif quarter == "Q2": 
    tgt_date = f"{tgt_year}-04-01"
elif quarter == "Q3": 
    tgt_date = f"{tgt_year}-07-01"
elif quarter == "Q4": 
    tgt_date = f"{tgt_year}-10-01"

tgt_date = datetime.datetime.strptime(tgt_date, "%Y-%m-%d")

print(tgt_date)

#//*** Get Filename for dataframe Cache
cache_filepath =  f"{tgt_year}_{quarter}_scripts.dat"
current_dir = Path(os.getcwd()).absolute()
cache_filepath = current_dir.joinpath(cache_filepath)

#//*** If Cache File Exists Load it. Else Start a Fresh DataFrame
if os.path.exists(cache_filepath):
    cache_df = pd.read_pickle(cache_filepath)
else:
    cache_df = pd.DataFrame()




2022-10-01 00:00:00


In [2]:
#//*** Parses a 
def get_scripts(source_urls,headless=True):
    
    out = []
    
    #//*** Initialize Headless Firefox options 
    options = webdriver.FirefoxOptions()
    options.add_argument('-headless')   
    
    if headless:


        #//*** Span Headless Instance
        page_driver = webdriver.Firefox(options=options)
    else:
        # run firefox webdriver from executable path of your choice
        page_driver = webdriver.Firefox()

    
    
    for i,url in enumerate(source_urls):
        
        error_counter = 1
        print(f"({i+1}/{len(source_urls)}) - {url}")
        
        while error_counter < 6:
            print("Downloading Page: Attempt ",error_counter )
            try:
                page_driver.get(url)
                error_counter = 10
            except:
                error_counter += 1
                
                print("Problem Getting Page...Trying Again")

        #//*** Reset any Story errors 
        story_error = False
        
        loop_story = {
                "url" : url,
                "headline" : None,
                "time_text" : None,
                "body_text" : None,
                'time_date' : None,
            }
        #//*******************
        #//**** Get Headline
        #//*******************

        #FITT_Article_main__body
        main_body = page_driver.find_elements(By.CLASS_NAME,"FITT_Article_main__body")
        
        #//*** Verify we have main_body elements
        if len(main_body) > 0:
            #//*** Assume Main Body is the first element. If not we in trouble
            #//*** mmmmm dirty dirty Type conversion
            main_body = main_body[0]
            

            #//*******************
            #//**** Get Headline
            #//*******************
            #//*** Only assign valid headline
            
            #//*** Get the Headline from the main Body. It should be the first (and only) h1. 
            headline = main_body.find_elements(By.CSS_SELECTOR,"h1")
            
            #//*** Check to ensure we actually have h1 elements
            if len(headline) > 0:
                #//*** Dirty dirty type conversion
                headline = headline[0].text
                
                #//*** Assign Headline
                loop_story["headline"] = headline
        
            #//********************
            #//**** Get Date
            #//********************

            #//*** Date is located withing the ShareByline Class
            byLine = main_body.find_elements(By.CLASS_NAME,"ShareByline")[0]

            #//*** Get All divs under ShareByline
            div_list = byLine.find_elements(By.CSS_SELECTOR,"div")

            #//*** The Date will be in a div with no children
            for elem in div_list:
                #//*** If div has div children, skip it
                if len(elem.find_elements(By.CSS_SELECTOR,"div")) > 0:
                       continue

                #//*** Try to convert Element Text to Date
                #//*** Skip Element if conversion fails
                try:
                    d = pd.to_datetime(elem.text)        
                except:
                    continue

                #//*** Verify d is actually a timestamp / Date.
                #//*** Will get NaT result on empty field
                if isinstance(d,pd._libs.tslibs.timestamps.Timestamp):
                    loop_story["time_text"] = elem.text
                    loop_story['time_date'] = d
                    break

            #//********************************
            #//*** Get Main Body Text
            #//********************************

            #//*** Article is contained within the article tag
            article = main_body.find_elements(By.CSS_SELECTOR,"article")

            whole_text = ""

            #//*** Double check we actually have the article
            if len(article) > 0:
                #//*** We could take the entire article text, but it includes related links and add text.
                #//*** We are better than that.
                #//*** Get each p element. Keep elements that have 0 children
                #//*** In the example, the first p element has one child, probably for datelineing. We'll keep the first
                #//*** p element then skip any element that has children.

                elems = article[0].find_elements(By.CSS_SELECTOR,"p")

                #//*** Check to see if we actually have story text
                if len(elems) > 0:
                    #//*** Assume First elem is valid
                    whole_text += elems[0].text
                    for elem in elems:
                        try:
                            #//*** Only Keep elements with no children
                            if len(elem.find_elements(By.XPATH, ".//*")) == 0:
                                whole_text += elem.text
                        except:
                            print("Problem processing Story Text. Element may have gone stale")
                            story_error = True 

                    loop_story["body_text"] = whole_text
            else:
                pass

            

            
        else:
            print("Problem Processing Main Body. Got a core parsing issue")
            story_error = True
    
        
        #//*** Throw an error if any loop_story field is None:
        for key,value in loop_story.items():
            
            if value == None:
                story_error = True
                print("Story Parsing Error:",key,"is",value)
            
        #//*** Keep Story if No Errors
        if not story_error:
            out.append(loop_story)
        
        #wait between 1/2 to 3 seconds
        time.sleep(random.randint(500, 3000)/1000)                                                              

    page_driver.quit()
                                                                        
    return out

In [3]:
def get_all_urls_in_page(source_url,tgt_date,headless=True):

    #//*** Keep all classnames in a dictionary
    g = {
        #//*** Classname for the Show More Button
        "class_more_button" : "show-button-more",
        
        #//*** Grid Classname that contains the linked stories
        "story_grid" : "grid3",
        
        #//*** Story Classes that contain the href Links
        "story_link_class" : "AnchorLink",
        
    }
    
    #//*** Initialize Headless Firefox options 
    options = webdriver.FirefoxOptions()
    options.add_argument('-headless')   
    
    if headless:


        #//*** Span Headless Instance
        driver = webdriver.Firefox(options=options)
    else:
        # run firefox webdriver from executable path of your choice
        driver = webdriver.Firefox()


    # Load Web Page
    driver.get(source_url)
    
    # execute script to scroll down the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")


    #//*****************************************************
    #//*** Wait until the Show More Button is loaded
    #//*****************************************************
    elements = []
    control = 0
    while len(elements) == 0:

        #//*** Page is Loaded when class_more_button is displayed
        #elements = driver.find_elements_by_class_name(g["class_more_button"])
        elements = driver.find_elements(By.CLASS_NAME,g["class_more_button"])

        control += 1
        #//*** Wait if element not found
        if len(elements) == 0:
            time.sleep(1)

        if control > 15:
            print("Show More Button Not Loaded!")
            return
            break

    #//*************************************************
    #//*** The first elements is the button to click
    #//*************************************************
    elem_show_more = elements[0]

    #actions = ActionChains(driver)
    #actions.move_to_element(element).perform()

    print(elem_show_more.is_displayed())
    print(elem_show_more.location['y'])
    
    #results = driver.find_elements_by_class_name(g["story_grid"])[0].find_elements_by_class_name(g["story_link_class"])
    results = driver.find_elements(By.CLASS_NAME,g["story_grid"])[0].find_elements(By.CLASS_NAME,g["story_link_class"])

    if len(results) > 0:
        print("Last Page: ", results[0].get_property('href'))

    
    
    #print(results[-1].get_property('href'))

    
    #print(story_time,tgt_date, story_time < tgt_date)
                                                                        
    #//*** Get the last url to check in stories are within the quarter
    last_url = results[-1].get_property('href')
    
    try:
        #//*** Download the last script
        last_script = get_scripts([last_url])[0]
    except:
        print("Problem getting last script")
        print("Last url: ", last_url)
        print("get Scripts: ", get_scripts([last_url]))
        print("Skipping..")
        return []
    last_date = last_script['time_date']
    print("Last Script: ", last_script['headline'],"\n",last_script["time_text"])
    print("Last Script: ", last_script['time_date'] < tgt_date)
    
    #//*** Maximum number of while loops.
    maxDepth = 20
    depth = 0
    
    #//*** Click Show More until Last Story is older than the tgt_date
    while last_date > tgt_date:
        depth += 1
        
        #//*** Scroll to Bottom of page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        
        print(f"Clicking More Depth: {depth}")
        #print(dir(elem_show_more))
        while elem_show_more.is_displayed() == False:
            print("Is Displayed:",str(elem_show_more.is_displayed()))
            time.sleep(1)
        
        
        try:
            #//*** Click Show More
            elem_show_more.click()
        except:
            iframes = driver.find_elements_by_tag_name("iframe")
            for iframe in iframes:
                print(iframe.get_property("name"))
                
                
                driver.execute_script("arguments[0].style.visibility='hidden'", iframe)
            #//*** Click Show More
            elem_show_more.click()
                
        #    return

    
        #//*** Get Updated Results
        #results = driver.find_elements_by_class_name(g["story_grid"])[0].find_elements_by_class_name(g["story_link_class"])        
        results = driver.find_elements(By.CLASS_NAME,g["story_grid"])[0].find_elements(By.CLASS_NAME,g["story_link_class"])        
        
        #//*** Get the last url to check in stories are within the quarter
        last_url = results[-1].get_property('href')

        #//*** Download the last script
        script_list = get_scripts([last_url])
        if len(script_list) == 0:
            print("No Scripts Found in this category")
            print("Quitting")
            driver.quit()
            return []
            
        last_script = script_list[0]
        last_date = last_script['time_date']
        print("Last Script: ", last_script['headline'],"\n",last_script["time_text"])
        print("Last Script: ", last_script['time_date'] < tgt_date)

        if depth > maxDepth:
            print("Maximum Depth Reached on Clicking Show More")
            break
        
    
    out = []
    
    for result in results:
        out.append(result.get_property('href'))
    
    #//*** Shut down main scrape
    driver.quit()
    
    

    return out

In [4]:
#//*** Hash url list and compare to cache_df
#//*** returns a tuple of urls (orig_urls,dupe_urls)
def keep_original_urls(urls,cache_df):
    orig_urls = []
    dupe_urls = []
    
    tdf = pd.DataFrame()
    tdf['urls'] = urls
    
    tdf['hash'] = pd.util.hash_pandas_object(tdf['urls'])


    #//*** Check for Duplicates
    tdf['dupe'] = tdf['hash'].isin(cache_df['hash'])
    #tdf['dupe'] = tdf['urls'].isin(cache_df['urls'])
    
    #//*** Keep Only False Stories, which are not duplicates
    #//*** This means a story can have only a single category.
    #//*** May have to handle multiple categories at a future time
    #print(f"Length Before: {len(tdf)} ")

    orig_urls = list(tdf[tdf['dupe'] == False]['urls'])
    dupe_urls = list(tdf[tdf['dupe'] == True]['urls'])

    
    
    return orig_urls,dupe_urls

In [5]:

url_items = [
    {
        "url" : "https://abc7news.com/tag/climate-change/",
        "cat" : "Climate"
    },
    {
        "url" : "https://abc7news.com/tag/environment/",
        "cat" : "Environment"
    },
 
    {
        "url" : "https://abc7news.com/tag/economy/",
        "cat" : "Economy"
    },
    {
        "url" : "https://abc7news.com/tag/jobs/",
        "cat" : "Jobs"
    },    
    {
        "url" : "https://abc7news.com/tag/safety/",
        "cat" : "Safety"
    },
    {
        "url" : "https://abc7news.com/tag/health/",
        "cat" : "Health"
    },
    {
        "url" : "https://abc7news.com/education/",
        "cat" : "Education"
    },
    {
        "url" : "https://abc7news.com/tag/race-and-culture/",
        "cat" : "Race"
    },
    {
        "url" : "https://abc7news.com/tag/discrimination/",
        "cat" : "Discrimination"
    },
    {
        "url" : "https://abc7news.com/tag/civil-rights/",
        "cat" : "Civil Rights"
    },
    {
        "url" : "https://abc7news.com/tag/building-a-better-bay-area/",
        "cat" : "BABBA"
    },
    {
        "url" : "https://abc7news.com/7onyourside/",
        "cat" : "7OYS"
    },
    {
        "url" : "https://abc7news.com/tag/covid-19/",
        "cat" : "COVID"
    },

    {
        "url" : "https://abc7news.com/iteam/",
        "cat" : "I-TEAM"
    },
    

    
]
urlpages = ['https://abc7news.com/education/',
            'https://abc7news.com/tag/building-a-better-bay-area/',
            'https://abc7news.com/7onyourside/',
            'https://abc7news.com/tag/covid-19/',
            'https://abc7news.com/tag/climate-change/',
            'https://abc7news.com/iteam/'
           ]

#urlpage ='https://abc7news.com/education/'
#urls = get_all_urls_in_page(urlpages[1],tgt_date,False)
#print(urlpage)

#//*** Loop through top level url_items
for item in url_items:
    cat_url = item['url']
    cat = item['cat']
    
    #//*** Download the ctageory URLs
    urls = get_all_urls_in_page(cat_url,tgt_date,False)
    
    #//*** Get non-duplicate urls to download and a list of the duplicate urls
    #//*** We may need to incorporate duplicates with different categories...later
    if len(cache_df) > 0:
        urls_to_dl,dupe_urls = keep_original_urls(urls,cache_df)
    else:
        #//*** There is no cache initialize and move one
        urls_to_dl = urls
        dupe_urls = []
    print("=====================")
    print(f"Category: {cat}")
    print("=====================")
    print(f"DL Items: {len(urls_to_dl)} - Duplicate: {len(dupe_urls)}")
    
    if len(urls_to_dl) == 0:
        print("No New Items to Download!")
    
    if len(urls_to_dl) > 0:
        scripts = get_scripts(urls_to_dl)
        
        #//*** Convert Scripts to DataFrame
        df = pd.DataFrame()
        #df['urls'] = urls_to_dl

        s = {}

        #//*** Get each script value as a list
        for script in scripts:

            for key,value in script.items():
                if key not in s.keys():
                    s[key] = []
                s[key].append(value)

        #//*** build Urls field
        #//*** Use the Story URL in case an url was skipped.
        df['urls'] = s['url']
        df['hash'] = pd.util.hash_pandas_object(df['urls'])
        df['cat'] = cat

        #//*** Add lists of script values as columns
        for key,value in s.items():
            df[key] = value

        #//*** Combine cache_df and df
        cache_df = pd.concat([cache_df,df])


#//*** Everything is Gathered. Write Cache_df to disk
#//*** Write DF to file
pd.to_pickle(cache_df,cache_filepath)
    

True
1830
Last Page:  https://abc7news.com/nasa-mission-earth-water-california-drought-satellite-surface-and-ocean-topography/12428070/
(1/1) - https://abc7news.com/design-by-fire-emily-schlickman-brett-milligan-california-wildfires-wildfire-prevention/12301384/
Last Script:  New book by UC Davis researchers documents CA wildfire strategies, hopes to aid future prevention 
 Thursday, October 13, 2022
Last Script:  False
Clicking More Depth: 1
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
(1/1) - https://abc7news.com/power-outage-ca-heat-wave-flex-alert-california-schools-ac-energy-use-weather/12211859/
Last Script:  CA enters 10th day of Flex Alert, last day of Bay Area heat wave 
 Friday, September 9, 2022
Last Script:  True
Category: Climate
DL Items: 24 - Duplicate: 0
(1/24) - https://abc7new

(6/36) - https://abc7news.com/shop-safe-oakland-holiday-shopping-small-businesses-safety/12487095/
(7/36) - https://abc7news.com/san-jose-aloha-downtown-skate-rink-holiday-roller-sj-small-businesses-circle-of-palms-plaza/12483939/
(8/36) - https://abc7news.com/women-and-money-finance-financial-literacy-sequin/12481907/
(9/36) - https://abc7news.com/ftx-crypto-collapse-news/12476302/
(10/36) - https://abc7news.com/san-francisco-tourism-sfo-international-travelers-asian-countries-covid-travel-restrictions-lifted-korea/12461961/
(11/36) - https://abc7news.com/ftx-crypto-lawsuit-tom-brady/12459754/
(12/36) - https://abc7news.com/california-budget-deficit-gavin-newsom-ca-taxes/12460121/
(13/36) - https://abc7news.com/rail-strike-november-2022-railroad-unions-brotherhood-of-boilermakers-freight-trains/12455671/
(14/36) - https://abc7news.com/mba-recruiting-northwestern-kellogg-school-of-business-testing-waived-laid-off-tech-workers/12454705/
(15/36) - https://abc7news.com/tech-layoff-bay-are

Story Parsing Error: time_text is None
Story Parsing Error: time_date is None
(10/11) - https://abc7news.com/halloween-2022-parade-pennsylvania-lower-merion-school-district/12314481/
(11/11) - https://abc7news.com/fisher-price-rock-n-play-sleeper-inclined-recall-deaths-infant-injured-by/12276822/
True
1797
Last Page:  https://abc7news.com/raw-oysters-fda-recall-dai-one-food-company/12494711/
(1/1) - https://abc7news.com/california-proposition-29-2022-dialysis-clinic-prop-election/12295409/
Last Script:  Proposition 29: Californians reject measure to alter dialysis clinic rules, AP projects 
 Wednesday, November 9, 2022
Last Script:  False
Clicking More Depth: 1
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
(1/1) - https://abc7news.com/covid-19-booster-omicron-variant-strains-vaccine-bay-area/12220344/
Last Script:  New COVID-19 booste

(17/36) - https://abc7news.com/ebike-boom-young-riders-electric-bike-safety-concerns-marin-county-bicycle-coalition/12353151/
(18/36) - https://abc7news.com/2023-disney-dreamers-academy-application-high-school-students-career/12351027/
(19/36) - https://abc7news.com/san-jose-state-university-tommie-smith-john-carlos-sjsu-track-and-field/12325240/
(20/36) - https://abc7news.com/no-school-photos-hayward-twin-oaks-montessori-kid-pix-pictures/12325543/
(21/36) - https://abc7news.com/hbcu-week-historically-black-college-and-university-walt-disney-world-resort-espn-wide-of-sports-complex/12309119/
(22/36) - https://abc7news.com/nicki-minaj-the-black-barbie-femmecee-and-hip-hop-feminisms-uc-berkeley-college-course/12309237/
(23/36) - https://abc7news.com/hbcu-week-disney-2022-magic-kingdom-college-marching-bands/12306425/
(24/36) - https://abc7news.com/hbcu-week-historically-black-collage-and-university-walt-disney-world-resort-college-scholarship/12303415/
(25/36) - https://abc7news.com/hisp

Last Script:  Malcolm X's daughter, Malikah Shabazz, found dead in Brooklyn home 
 Tuesday, November 23, 2021
Last Script:  True
Category: Civil Rights
DL Items: 2 - Duplicate: 10
(1/2) - https://abc7news.com/montclair-mr-gill-school-history/11955461/
Story Parsing Error: time_text is None
Story Parsing Error: time_date is None
(2/2) - https://abc7news.com/malcolm-x-daughter-dead-malikah-shabazz-obit/11263607/
True
1797
Last Page:  https://abc7news.com/gift-4-martinez-card-small-business-holiday-shopping/12495570/
(1/1) - https://abc7news.com/sf-muni-central-subway-grand-opening-chinatown-art/12473143/
Last Script:  'This is big': What it's like to ride SF Muni's new Central Subway; Unofficial opening this weekend 
 Sunday, November 20, 2022
Last Script:  False
Clicking More Depth: 1
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
(1/1) - https://abc7news.

(44/58) - https://abc7news.com/alas-half-moon-bay-area-farmers-services-for-farmworkers-retrofitted-bus/12324368/
(45/58) - https://abc7news.com/san-jose-state-university-tommie-smith-john-carlos-sjsu-track-and-field/12325240/
(46/58) - https://abc7news.com/sf-bike-lanes-lane-bounty-reward-for-reporting-double-parking-drivers-on/12329918/
(47/58) - https://abc7news.com/stanford-brain-research-human-rat-study-transplanting-cells-into-rats-cell/12321268/
(48/58) - https://abc7news.com/san-francisco-metro-area-census-bureau-data-affordable-housing-prices/12326094/
(49/58) - https://abc7news.com/sfpd-budget-defund-the-police-department-funding/12321818/
(50/58) - https://abc7news.com/grocery-stores-banning-single-use-plastic-bags-meat-produce-senate-bill-1046/12298432/
(51/58) - https://abc7news.com/oakland-crime-shooting-ghost-guns-police-chief-leronne-armstrong/12303237/
(52/58) - https://abc7news.com/covid-19-pandemic-hiv-cases-symptoms-and-aids/12301452/
(53/58) - https://abc7news.com/

Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
(1/1) - https://abc7news.com/taking-a-sick-day-flu-season-calling-in-from-work-months/12372401/
Last Script:  Feeling sick? Bay Area doctor suggests calling out from work this tough COVID, flu season 
 Monday, October 24, 2022
Last Script:  False
Clicking More Depth: 2
(1/1) - https://abc7news.com/moderna-lawsuit-pfizer-biontech/12168319/
Last Script:  Moderna to sue BioNTech/Pfizer over COVID vaccine 
 Friday, August 26, 2022
Last Script:  True
Category: COVID
DL Items: 35 - Duplicate: 1
(1/35) - https://abc7news.com/paxlovid-drug-long-covid-stanford-study-brain-fog/12483220/
(2/35) - https://abc7news.com/goldendoodle-bay-area-puppy-therapy-family-loss-covid-19-east-palo-alto-dillon-doodles-vacaville/12469709/
(3/35) - https://abc7news.com/pink-opens-up-good-morning-america-interview-trus

(15/24) - https://abc7news.com/pge-fire-settlement-california-wildfire-money-tax-free-federal-for/12321178/
(16/24) - https://abc7news.com/defunding-the-police-sjpd-budget-san-jose-increase-use-of-force/12316493/
(17/24) - https://abc7news.com/defund-police-oakland-crime-shooting/12311750/
(18/24) - https://abc7news.com/california-wildfires-pge-stocksca-fire-victim-trust-ca-wildfire-settlement-victims/12300992/
(19/24) - https://abc7news.com/deer-at-39th-avenue-reservoir-mr-buck-oakland-trapped-behind-fence-animal-rescue/12298501/
(20/24) - https://abc7news.com/napoleon-brown-london-breed-brother-prison-sentence-district-attorney-brooke-jenkins/12292426/
(21/24) - https://abc7news.com/caltrans-traffic-road-conditions-highway-101-san-mateo-county-express-lane-project/12283716/
(22/24) - https://abc7news.com/oakland-school-shooting-update-rudsdale-continuation-high-fontaine-street-king-estate-campus/12279258/
(23/24) - https://abc7news.com/oakland-ca-school-shooting-rudsdale-king-estates

In [6]:
cache_df

,urls,hash,cat,url,headline,time_text,body_text,time_date
0,https://abc7news.com/whales-san-francisco-bay-...,2112476080608435129,Climate,https://abc7news.com/whales-san-francisco-bay-...,SF Bay whale migration patterns put priority o...,"Friday, October 14, 2022",SAN FRANCISCO (KGO) -- As a researcher at the ...,2022-10-14
1,https://abc7news.com/whales-san-francisco-bay-...,13918837101855457112,Climate,https://abc7news.com/whales-san-francisco-bay-...,SF Bay whale migration patterns put priority o...,"Friday, October 14, 2022",SAN FRANCISCO (KGO) -- As a researcher at the ...,2022-10-14
2,https://abc7news.com/design-by-fire-emily-schl...,9351435183199199193,Climate,https://abc7news.com/design-by-fire-emily-schl...,New book by UC Davis researchers documents CA ...,"Thursday, October 13, 2022",SAN FRANCISCO (KGO) -- For victims of Californ...,2022-10-13
3,https://abc7news.com/san-francisco-sea-levels-...,1839742217112731968,Climate,https://abc7news.com/san-francisco-sea-levels-...,"Sea levels could rise 7 feet by 2100, official...","Wednesday, October 12, 2022",SAN FRANCISCO (KGO) -- San Francisco is launch...,2022-10-12
4,https://abc7news.com/ghost-boat-shasta-lake-dr...,18337610589246583208,Climate,https://abc7news.com/ghost-boat-shasta-lake-dr...,Mystery surrounds WWII-era 'ghost boat' found ...,"Tuesday, October 11, 2022","REDDING, Calif. -- A World War II-era boat was...",2022-10-11
...,...,...,...,...,...,...,...,...
18,https://abc7news.com/deer-at-39th-avenue-reser...,14881517701333071758,I-TEAM,https://abc7news.com/deer-at-39th-avenue-reser...,EXCLUSIVE: Oakland neighbors worry about young...,"Thursday, October 6, 2022","OAKLAND, Calif. (KGO) -- We've all been dealin...",2022-10-06
19,https://abc7news.com/napoleon-brown-london-bre...,13248276614306900914,I-TEAM,https://abc7news.com/napoleon-brown-london-bre...,SF mayor's brother appears in court to get 44-...,"Monday, October 3, 2022",SAN FRANCSICO (KGO) -- The brother of San Fran...,2022-10-03
20,https://abc7news.com/caltrans-traffic-road-con...,98100517902407247,I-TEAM,https://abc7news.com/caltrans-traffic-road-con...,ABC7 I-Team looks into express lane delays on ...,"Friday, September 30, 2022",SAN FRANCISCO (KGO) -- Drivers who use Highway...,2022-09-30
21,https://abc7news.com/oakland-school-shooting-u...,1569790983728564152,I-TEAM,https://abc7news.com/oakland-school-shooting-u...,Oakland school shooting: New cellphone video s...,"Friday, September 30, 2022","OAKLAND, Calif. (KGO) -- All new cellphone vid...",2022-09-30


In [7]:
#cache_df['time_date'] = cache_df['time_date'].dt.date
#print("Length Before de-dupe:",len(cache_df)
#cache_df = cache_df.drop_duplicates(subset="url")
      
#print("Length Before de-dupe:",len(cache_df)

cache_df.drop_duplicates(subset="url")
cache_df.drop_duplicates(subset="headline")

,urls,hash,cat,url,headline,time_text,body_text,time_date
0,https://abc7news.com/whales-san-francisco-bay-...,2112476080608435129,Climate,https://abc7news.com/whales-san-francisco-bay-...,SF Bay whale migration patterns put priority o...,"Friday, October 14, 2022",SAN FRANCISCO (KGO) -- As a researcher at the ...,2022-10-14
2,https://abc7news.com/design-by-fire-emily-schl...,9351435183199199193,Climate,https://abc7news.com/design-by-fire-emily-schl...,New book by UC Davis researchers documents CA ...,"Thursday, October 13, 2022",SAN FRANCISCO (KGO) -- For victims of Californ...,2022-10-13
3,https://abc7news.com/san-francisco-sea-levels-...,1839742217112731968,Climate,https://abc7news.com/san-francisco-sea-levels-...,"Sea levels could rise 7 feet by 2100, official...","Wednesday, October 12, 2022",SAN FRANCISCO (KGO) -- San Francisco is launch...,2022-10-12
4,https://abc7news.com/ghost-boat-shasta-lake-dr...,18337610589246583208,Climate,https://abc7news.com/ghost-boat-shasta-lake-dr...,Mystery surrounds WWII-era 'ghost boat' found ...,"Tuesday, October 11, 2022","REDDING, Calif. -- A World War II-era boat was...",2022-10-11
5,https://abc7news.com/west-coast-climate-agreem...,7989243367173598296,Climate,https://abc7news.com/west-coast-climate-agreem...,"Governor Newsom, West Coast leaders sign clima...","Thursday, October 6, 2022",SAN FRANCISCO (KGO) -- Governor Gavin Newsom w...,2022-10-06
...,...,...,...,...,...,...,...,...
5,https://abc7news.com/santa-clara-county-sherif...,211859092911097547,I-TEAM,https://abc7news.com/santa-clara-county-sherif...,EXCLUSIVE: Interview with Laurie Smith followi...,"Monday, October 31, 2022","SAN JOSE, Calif. (KGO) -- The Santa Clara Coun...",2022-10-31
6,https://abc7news.com/oakland-youth-sports-park...,8725094034212069941,I-TEAM,https://abc7news.com/oakland-youth-sports-park...,Oakland youth sports teams question where mone...,"Friday, October 28, 2022","OAKLAND, Calif. (KGO) -- Oakland parents and t...",2022-10-28
7,https://abc7news.com/atherton-car-buried-calif...,18915904937746550,I-TEAM,https://abc7news.com/atherton-car-buried-calif...,No body found in car buried at $15M Atherton m...,"Monday, October 24, 2022","ATHERTON, Calif. (KGO) -- Detectives finished ...",2022-10-24
8,https://abc7news.com/petit-four-films-wedding-...,13965584634779413840,I-TEAM,https://abc7news.com/petit-four-films-wedding-...,Couples stranded on wedding day say they're st...,"Friday, October 21, 2022",SAN FRANCISCO (KGO) -- Couples stranded by a v...,2022-10-21


In [8]:
#//*** Write to excel file
output_filename = f"{tgt_year}_{quarter}_Collected_Stories.xlsx"
print(output_filename)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(output_filename, engine='xlsxwriter')

#//*** Build Sheet and Data from From each url_item element
for url_item in url_items:
    sheet = url_item['cat']
    
    tdf = cache_df[cache_df['cat'] == sheet].copy()
    
    tdf = tdf[['urls','time_date','time_text','headline','body_text']].sort_values('time_date',ascending=False)
    
    tdf.to_excel(writer,sheet_name=sheet)
    
try:
    writer.save()    
except:
    print("Trouble Saving the Spreadsheet. It's Probably Open Somewhere. Close it and Retry")    
    

2022_Q4_Collected_Stories.xlsx


In [9]:
cache_df

,urls,hash,cat,url,headline,time_text,body_text,time_date
0,https://abc7news.com/president-joe-biden-clima...,3340588433118293791,Climate,https://abc7news.com/president-joe-biden-clima...,President Biden to announce new actions on cli...,"Wednesday, July 20, 2022",WASHINGTON -- President Joe Biden will announc...,2022-07-20
1,https://abc7news.com/president-joe-biden-clima...,11179669141244742564,Climate,https://abc7news.com/president-joe-biden-clima...,Biden holds off - for now - on climate emergen...,"Tuesday, July 19, 2022",WASHINGTON -- President Joe Biden will travel ...,2022-07-19
2,https://abc7news.com/indigenous-forest-managem...,9544418966223491738,Climate,https://abc7news.com/indigenous-forest-managem...,Indigenous peoples use ceremonial burns as Red...,"Friday, July 15, 2022","SANTA CRUZ, Calif. (KGO) -- As a senior land m...",2022-07-15
3,https://abc7news.com/supreme-court-remaining-c...,4614620187172708451,Climate,https://abc7news.com/supreme-court-remaining-c...,Supreme Court ruling limits EPA's ability to r...,"Thursday, June 30, 2022",WASHINGTON -- The Supreme Court on Thursday li...,2022-06-30
4,https://abc7news.com/air-quality-bay-area-spar...,11398640325941118735,Climate,https://abc7news.com/air-quality-bay-area-spar...,No Spare the Air Alert issued in the Bay Area,"Wednesday, June 22, 2022",SAN FRANCISCO -- There is no Spare the Air Ale...,2022-06-22
...,...,...,...,...,...,...,...,...
30,https://abc7news.com/ivf-treatment-ro-v-wade-a...,15306288454229551831,I-TEAM,https://abc7news.com/ivf-treatment-ro-v-wade-a...,Doctors say IVF treatment process likely to ge...,"Tuesday, June 28, 2022",SAN FRANCISCO (KGO) -- The overturn of Roe wil...,2022-06-28
31,https://abc7news.com/planned-parenthood-aborti...,9534249791868771498,I-TEAM,https://abc7news.com/planned-parenthood-aborti...,NorCal Planned Parenthood clinics ready for in...,"Tuesday, June 28, 2022",SAN FRANCISCO (KGO) -- People across the count...,2022-06-28
32,https://abc7news.com/daily-harvest-lentil-crum...,3952527858334626713,I-TEAM,https://abc7news.com/daily-harvest-lentil-crum...,"Daily Harvest responds to customers sickened, ...","Tuesday, June 28, 2022",SAN FRANCISCO (KGO) -- Meal delivery service D...,2022-06-28
33,https://abc7news.com/priest-sex-abuse-catholic...,11645930866639425220,I-TEAM,https://abc7news.com/priest-sex-abuse-catholic...,Survivor of priest sex abuse speaks out as new...,"Monday, June 27, 2022",SAN FRANCISCO (KGO) -- New lawsuits are floodi...,2022-06-27


In [9]:
cache_df.drop_duplicates(subset="url")
cache_df.drop_duplicates(subset="headline")

,urls,hash,cat,url,headline,time_text,body_text,time_date
0,https://abc7news.com/whales-san-francisco-bay-...,2112476080608435129,Climate,https://abc7news.com/whales-san-francisco-bay-...,SF Bay whale migration patterns put priority o...,"Friday, October 14, 2022",SAN FRANCISCO (KGO) -- As a researcher at the ...,2022-10-14
2,https://abc7news.com/design-by-fire-emily-schl...,9351435183199199193,Climate,https://abc7news.com/design-by-fire-emily-schl...,New book by UC Davis researchers documents CA ...,"Thursday, October 13, 2022",SAN FRANCISCO (KGO) -- For victims of Californ...,2022-10-13
3,https://abc7news.com/san-francisco-sea-levels-...,1839742217112731968,Climate,https://abc7news.com/san-francisco-sea-levels-...,"Sea levels could rise 7 feet by 2100, official...","Wednesday, October 12, 2022",SAN FRANCISCO (KGO) -- San Francisco is launch...,2022-10-12
4,https://abc7news.com/ghost-boat-shasta-lake-dr...,18337610589246583208,Climate,https://abc7news.com/ghost-boat-shasta-lake-dr...,Mystery surrounds WWII-era 'ghost boat' found ...,"Tuesday, October 11, 2022","REDDING, Calif. -- A World War II-era boat was...",2022-10-11
5,https://abc7news.com/west-coast-climate-agreem...,7989243367173598296,Climate,https://abc7news.com/west-coast-climate-agreem...,"Governor Newsom, West Coast leaders sign clima...","Thursday, October 6, 2022",SAN FRANCISCO (KGO) -- Governor Gavin Newsom w...,2022-10-06
...,...,...,...,...,...,...,...,...
7,https://abc7news.com/deer-at-39th-avenue-reser...,10667358383279359763,I-TEAM,https://abc7news.com/deer-at-39th-avenue-reser...,EXCLUSIVE: Oakland neighbors worry about young...,"Thursday, October 6, 2022","OAKLAND, Calif. (KGO) -- We've all been dealin...",2022-10-06
8,https://abc7news.com/napoleon-brown-london-bre...,9456038266558042981,I-TEAM,https://abc7news.com/napoleon-brown-london-bre...,SF mayor's brother appears in court to get 44-...,"Monday, October 3, 2022",SAN FRANCSICO (KGO) -- The brother of San Fran...,2022-10-03
9,https://abc7news.com/caltrans-traffic-road-con...,17568493945191498898,I-TEAM,https://abc7news.com/caltrans-traffic-road-con...,ABC7 I-Team looks into express lane delays on ...,"Friday, September 30, 2022",SAN FRANCISCO (KGO) -- Drivers who use Highway...,2022-09-30
10,https://abc7news.com/oakland-school-shooting-u...,7656305319714397128,I-TEAM,https://abc7news.com/oakland-school-shooting-u...,Oakland school shooting: New cellphone video s...,"Friday, September 30, 2022","OAKLAND, Calif. (KGO) -- All new cellphone vid...",2022-09-30


In [9]:
url_items = [
    {
        "url" : "https://abc7news.com/tag/climate-change/",
        "cat" : "Climate"
    },
    {
        "url" : "https://abc7news.com/tag/environment/",
        "cat" : "Environment"
    },
 
    {
        "url" : "https://abc7news.com/tag/economy/",
        "cat" : "Economy"
    },
    {
        "url" : "https://abc7news.com/tag/jobs/",
        "cat" : "Jobs"
    },    
    {
        "url" : "https://abc7news.com/tag/safety/",
        "cat" : "Safety"
    },
    {
        "url" : "https://abc7news.com/tag/health/",
        "cat" : "Health"
    },
    {
        "url" : "https://abc7news.com/education/",
        "cat" : "Education"
    },
    {
        "url" : "https://abc7news.com/tag/race-and-culture/",
        "cat" : "Race"
    },
    {
        "url" : "https://abc7news.com/tag/discrimination/",
        "cat" : "Discrimination"
    },
    {
        "url" : "https://abc7news.com/tag/civil-rights/",
        "cat" : "Civil Rights"
    },
    {
        "url" : "https://abc7news.com/tag/building-a-better-bay-area/",
        "cat" : "BABBA"
    },
    {
        "url" : "https://abc7news.com/7onyourside/",
        "cat" : "7OYS"
    },
    {
        "url" : "https://abc7news.com/tag/covid-19/",
        "cat" : "COVID"
    },

    {
        "url" : "https://abc7news.com/iteam/",
        "cat" : "I-TEAM"
    },
    

    
]

try:
    cache_df['time_date'] = cache_df['time_date'].dt.date
except:
    pass

#//*** Write to excel file
output_filename = f"{tgt_year}_{quarter}_Collected_Stories.xlsx"
print(output_filename)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(output_filename, engine='xlsxwriter')

#//*** Build Sheet and Data from From each url_item element
for url_item in url_items:
    sheet = url_item['cat']
    
    tdf = cache_df[cache_df['cat'] == sheet].copy().sort_values("time_date",ascending=False)
    
    tdf = tdf[['urls','time_date','time_text','headline','body_text']]
    
    tdf.to_excel(writer,sheet_name=sheet)
    

workbook = writer.book


cell_format = workbook.add_format()


for url_item in url_items:
    sheet = url_item['cat']
    writer.sheets[sheet].set_column(2,2, 10)      
    writer.sheets[sheet].set_column(4,4, 20) 
    
for url_item in url_items:
    sheet = url_item['cat']
    cell_format.set_font_name('Arial')
    cell_format.set_font_size(12)


     
    writer.sheets[sheet].set_column('A:F', None, cell_format)  
    
    
try:
    writer.close()    
except:
    print("Trouble Saving the Spreadsheet. It's Probably Open Somewhere. Close it and Retry")   

2022_Q4_Collected_Stories.xlsx


In [67]:
dir(cell_format)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_escape_attributes',
 '_escape_data',
 '_format_key',
 '_get_align_properties',
 '_get_alignment_key',
 '_get_border_key',
 '_get_color',
 '_get_dxf_index',
 '_get_fill_key',
 '_get_font_key',
 '_get_format_key',
 '_get_protection_properties',
 '_get_xf_index',
 '_set_filehandle',
 '_set_xml_writer',
 '_to_string',
 '_xml_close',
 '_xml_data_element',
 '_xml_declaration',
 '_xml_empty_tag',
 '_xml_empty_tag_unencoded',
 '_xml_end_tag',
 '_xml_formula_element',
 '_xml_inline_string',
 '_xml_number_element',
 '_xml_rich_inline_string',
 '_xml_rich_si_element',
 '_xml_si_element',
 '_xml_start_tag',
 '_xml_start_tag_unencod

In [12]:
"""
END MAIN PROGRAM
Degbugging Code Below
"""

'\nEND MAIN PROGRAM\nDegbugging Code Below\n'

In [13]:
STOP_HERE

NameError: name 'STOP_HERE' is not defined

In [ ]:
urls = ['https://abc7news.com/california-drought-santa-cruz-county-seawater-intrusion-soquel-creek-water-district/11869614/']
get_scripts(urls,False)

In [ ]:
page_driver = webdriver.Firefox()
page_driver.get(urls[0])

In [ ]:
url = urls[0]
loop_story = {
    "url" : url,
    "headline" : page_driver.find_elements(By.CLASS_NAME,"headline")[0].text,
    "time_text" : page_driver.find_elements(By.CLASS_NAME,"lastmodified")[0].text,
    "body_text" : page_driver.find_elements(By.CLASS_NAME,"body-text")[0].text,
}

In [ ]:
element = page_driver.find_element_by_tag_name("body")
raw_text = element.get_attribute('innerHTML')
for x in raw_text.split(">"):
    print(x)

In [ ]:
#//*******************
#//**** Get Headline
#//*******************

#FITT_Article_main__body
main_body = page_driver.find_elements(By.CLASS_NAME,"FITT_Article_main__body")[0]

#//*** Main Body Headline
main_body.find_elements(By.CSS_SELECTOR,"h1")[0].text


    

In [ ]:
#//********************
#//**** Get Date
#//********************

#//*** Date is located withing the ShareByline Class
byLine = main_body.find_elements(By.CLASS_NAME,"ShareByline")[0]
 
#//*** Get All divs under ShareByline
div_list = byLine.find_elements(By.CSS_SELECTOR,"div")

#//*** The Date will be in a div with no children
for elem in div_list:
    #//*** If div has div children, skip it
    if len(elem.find_elements(By.CSS_SELECTOR,"div")) > 0:
           continue
            
    #//*** Try to convert Element Text to Date
    #//*** Skip Element if conversion fails
    try:
        d = pd.to_datetime(elem.text)        
    except:
        continue
    
    #//*** Verify d is actually a timestamp / Date.
    #//*** Will get NaT result on empty field
    if isinstance(d,pd._libs.tslibs.timestamps.Timestamp):
        print("Date is ",d)
        break
    else:
        d = None
    

In [ ]:
#//*** Extract Main Body.

#//*** Article is contained within the article tag
article = main_body.find_elements(By.CSS_SELECTOR,"article")

whole_text = ""

#//*** Double check we actually have the article
if len(article) > 0:
    #//*** We could take the entire article text, but it includes related links and add text.
    #//*** We are better than that.
    #//*** Get each p element. Keep elements that have 0 children
    #//*** In the example, the first p element has one child, probably for datelineing. We'll keep the first
    #//*** p element then skip any element that has children.
    
    elems = article[0].find_elements(By.CSS_SELECTOR,"p")
    
    #//*** Assume First elem is valid
    whole_text += elems[0].text
    for elem in elems:
        #//*** Only Keep elements with no children
        if len(elem.find_elements(By.XPATH, ".//*")) == 0:
            whole_text += elem.text
else:
    pass

print(whole_text)

In [ ]:
for item in url_items:
    cat_url = item['url']
    cat = item['cat']
    
    #//*** Download the ctageory URLs
    urls = get_all_urls_in_page(cat_url,tgt_date,False)
    
    break

In [ ]:
#//*** Parses a 
def get_scripts(source_urls,headless=True):
    
    out = []
    
    #//*** Initialize Headless Firefox options 
    options = webdriver.FirefoxOptions()
    options.add_argument('-headless')   
    
    if headless:


        #//*** Span Headless Instance
        page_driver = webdriver.Firefox(options=options)
    else:
        # run firefox webdriver from executable path of your choice
        page_driver = webdriver.Firefox()

    
    
    for i,url in enumerate(source_urls):
        
        print(f"({i+1}/{len(source_urls)}) - {url}")
        try:
            page_driver.get(url)
        except:
            print("Problem Getting Page...Skipping")
            continue
        
        #//*** Reset any Story errors 
        story_error = False
        
        loop_story = {
                "url" : url,
                "headline" : None,
                "time_text" : None,
                "body_text" : None,
            }
        #//*******************
        #//**** Get Headline
        #//*******************

        #FITT_Article_main__body
        main_body = page_driver.find_elements(By.CLASS_NAME,"FITT_Article_main__body")
        
        #//*** Verify we have main_body elements
        if len(main_body) > 0:
            #//*** Assume Main Body is the first element. If not we in trouble
            #//*** mmmmm dirty dirty Type conversion
            main_body = main_body[0]
            

            #//*******************
            #//**** Get Headline
            #//*******************
            #//*** Only assign valid headline
            
            #//*** Get the Headline from the main Body. It should be the first (and only) h1. 
            headline = main_body.find_elements(By.CSS_SELECTOR,"h1")
            
            #//*** Check to ensure we actually have h1 elements
            if len(headline) > 0:
                #//*** Dirty dirty type conversion
                headline = headline[0].text
                
                #//*** Assign Headline
                loop_story["headline"] = headline
        
            #//********************
            #//**** Get Date
            #//********************

            #//*** Date is located withing the ShareByline Class
            byLine = main_body.find_elements(By.CLASS_NAME,"ShareByline")[0]

            #//*** Get All divs under ShareByline
            div_list = byLine.find_elements(By.CSS_SELECTOR,"div")

            #//*** The Date will be in a div with no children
            for elem in div_list:
                #//*** If div has div children, skip it
                if len(elem.find_elements(By.CSS_SELECTOR,"div")) > 0:
                       continue

                #//*** Try to convert Element Text to Date
                #//*** Skip Element if conversion fails
                try:
                    d = pd.to_datetime(elem.text)        
                except:
                    continue

                #//*** Verify d is actually a timestamp / Date.
                #//*** Will get NaT result on empty field
                if isinstance(d,pd._libs.tslibs.timestamps.Timestamp):
                    loop_story["time_text"] = d
                    break

            #//********************************
            #//*** Get Main Body Text
            #//********************************

            #//*** Article is contained within the article tag
            article = main_body.find_elements(By.CSS_SELECTOR,"article")

            whole_text = ""

            #//*** Double check we actually have the article
            if len(article) > 0:
                #//*** We could take the entire article text, but it includes related links and add text.
                #//*** We are better than that.
                #//*** Get each p element. Keep elements that have 0 children
                #//*** In the example, the first p element has one child, probably for datelineing. We'll keep the first
                #//*** p element then skip any element that has children.

                elems = article[0].find_elements(By.CSS_SELECTOR,"p")

                #//*** Assume First elem is valid
                whole_text += elems[0].text
                for elem in elems:
                    #//*** Only Keep elements with no children
                    if len(elem.find_elements(By.XPATH, ".//*")) == 0:
                        whole_text += elem.text
                
                loop_story["body_text"] = whole_text
            else:
                pass

            

            
        else:
            print("Problem Processing Main Body. Got a core parsing issue")
            story_error = True
        
        loop_story["body_text"] = None
        
        #//*** Throw an error if any loop_story field is None:
        for key,value in loop_story.items():
            
            if value == None:
                story_error = True
                print("Story Parsing Error:",key,"is",value)
            
        #//*** Keep Story if No Errors
        if not story_error:
            out.append(loop_story)
        
        #wait between 1/2 to 3 seconds
        time.sleep(random.randint(500, 3000)/1000)                                                              

    page_driver.quit()
                                                                        
    return out

scripts = get_scripts(['https://abc7news.com/president-joe-biden-climate-change-executive-actions-emergency-declaration/12062764/'])
print(scripts[0])
print('done')

In [ ]:

#//*** Split the Date
loop_story['time_date'] = loop_story['time_text'].split()

#//*** Rebuild Date string using only the first four fields. Drop the extra time
loop_story['time_date'] = " ".join(loop_story['time_date'][:4])

loop_story['time_date'] = datetime.datetime.strptime(loop_story['time_date'], "%A, %B %d, %Y")

In [ ]:
#//*** Backup Copy of original get_scripts
"""
#//*** Parses a 
def get_scripts(source_urls,headless=True):
    
    out = []
    
    #//*** Initialize Headless Firefox options 
    options = webdriver.FirefoxOptions()
    options.add_argument('-headless')   
    
    if headless:
    if headless:


        #//*** Span Headless Instance
        page_driver = webdriver.Firefox(options=options)
    else:
        # run firefox webdriver from executable path of your choice
        page_driver = webdriver.Firefox()

    
    
    for i,url in enumerate(source_urls):
        
        print(f"({i+1}/{len(source_urls)}) - {url}")
        try:
            page_driver.get(url)
        except:
            print("Problem Getting Page...Skipping")
            continue
            
        
        try:
        
            loop_story = {
                "url" : url,
                "headline" : page_driver.find_elements(By.CLASS_NAME,"headline")[0].text,
                "time_text" : page_driver.find_elements(By.CLASS_NAME,"lastmodified")[0].text,
                "body_text" : page_driver.find_elements(By.CLASS_NAME,"body-text")[0].text,
            }
            #//*** Split the Date
            loop_story['time_date'] = loop_story['time_text'].split()

            #//*** Rebuild Date string using only the first four fields. Drop the extra time
            loop_story['time_date'] = " ".join(loop_story['time_date'][:4])

            loop_story['time_date'] = datetime.datetime.strptime(loop_story['time_date'], "%A, %B %d, %Y")
        except:
            print("Problem Processing Story. Skipping")
            continue
        out.append(loop_story)
        
        #wait between 1/2 to 3 seconds
        time.sleep(random.randint(500, 3000)/1000)                                                              

    page_driver.quit()
                                                                        
    return out
"""
print()